In [32]:
N_ROUNDS = 11

In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import dataframe_image as dfi
from attacks import helper
import os
from pathlib import Path
pd.set_option('display.max_columns', None)

fmnist_b = pd.read_csv('data/wandb_export_2023-07-12_baseline_fashionmnist_f1.csv')
mnist_b = pd.read_csv('data/wandb_export_2023-07-12_baseline_mnist_f1.csv')
cifar10_b = pd.read_csv('data/wandb_export_2023-07-12_baseline_cifar10_f1.csv')

datasets = [mnist_b, fmnist_b, cifar10_b]

In [44]:
aggregation_list = ["Krum", "FedAvg", "TrimmedMean", "FlTrust", "Sentinel", "SentinelGlobal"]
aggregation_list.sort()
dataset_list = ["MNIST", "FashionMNIST", "CIFAR10"]
overview_error = pd.DataFrame(data=None, index=aggregation_list, columns=dataset_list, dtype=None, copy=None)

overview_error

,MNIST,FashionMNIST,CIFAR10
FedAvg,NaN,NaN,NaN
FlTrust,NaN,NaN,NaN
Krum,NaN,NaN,NaN
Sentinel,NaN,NaN,NaN
SentinelGlobal,NaN,NaN,NaN
TrimmedMean,NaN,NaN,NaN


In [45]:
for aggregator in aggregation_list:
    mnist_b_agg = mnist_b.filter(regex=(f'{aggregator}_'))
    mnist_max_agg_name = mnist_b_agg.filter(regex='__MAX').columns[0]
    mnist_min_agg_name = mnist_b_agg.filter(regex='__MIN').columns[0]
    mnist_b[f'{aggregator}_ERROR'] = mnist_b_agg[mnist_max_agg_name] - mnist_b_agg[mnist_min_agg_name]

for aggregator in aggregation_list:
    fmnist_b_agg = fmnist_b.filter(regex=(f'{aggregator}_'))
    fmnist_max_agg_name = fmnist_b_agg.filter(regex='__MAX').columns[0]
    fmnist_min_agg_name = fmnist_b_agg.filter(regex='__MIN').columns[0]
    fmnist_b[f'{aggregator}_ERROR'] = fmnist_b[fmnist_max_agg_name] - fmnist_b[fmnist_min_agg_name]

for aggregator in aggregation_list:
    cifar10_b_agg = cifar10_b.filter(regex=(f'{aggregator}_'))
    cifar10_max_agg_name = cifar10_b_agg.filter(regex='__MAX').columns[0]
    cifar10_min_agg_name = cifar10_b_agg.filter(regex='__MIN').columns[0]
    cifar10_b[f'{aggregator}_ERROR'] = cifar10_b[cifar10_max_agg_name] - cifar10_b[cifar10_min_agg_name]

In [46]:
mnist_f1 = mnist_b[mnist_b.columns.drop(list(mnist_b.filter(regex='__MAX')))]
mnist_f1 = mnist_f1[mnist_f1.columns.drop(list(mnist_f1.filter(regex='__MIN')))]

tmp = pd.DataFrame(index=range(0,N_ROUNDS), columns=aggregation_list)
for aggregator in aggregation_list:
    vals = mnist_f1.filter(regex=(f'{aggregator}_ERROR')).dropna()
    vals = vals.reset_index().iloc[:, 1]
    assert len(vals) == N_ROUNDS
    tmp[aggregator] = vals
tmp = tmp.reindex(sorted(tmp.columns), axis=1)
tmp = tmp.transpose()
overview_error['MNIST'] = tmp[10]

In [47]:
fmnist_f1 = fmnist_b[fmnist_b.columns.drop(list(fmnist_b.filter(regex='__MAX')))]
fmnist_f1 = fmnist_b[fmnist_b.columns.drop(list(fmnist_b.filter(regex='__MIN')))]

tmp = pd.DataFrame(index=range(0,N_ROUNDS), columns=aggregation_list)
for aggregator in aggregation_list:
    vals = fmnist_b.filter(regex=(f'{aggregator}_ERROR')).dropna()
    vals = vals.reset_index().iloc[:, 1]
    tmp[aggregator] = vals
    assert len(vals) == N_ROUNDS
tmp = tmp.reindex(sorted(tmp.columns), axis=1)
tmp = tmp.transpose()
overview_error['FashionMNIST'] = tmp[10]

In [48]:
cifar10_f1 = cifar10_b[cifar10_b.columns.drop(list(cifar10_b.filter(regex='__MAX')))]
cifar10_f1 = cifar10_f1[cifar10_f1.columns.drop(list(cifar10_f1.filter(regex='__MIN')))]

tmp = pd.DataFrame(index=range(0,N_ROUNDS), columns=aggregation_list)
for aggregator in aggregation_list:
    vals = cifar10_f1.filter(regex=(f'{aggregator}_ERROR')).dropna()
    vals = vals.reset_index().iloc[:, 1]
    assert len(vals) == N_ROUNDS
    tmp[aggregator] = vals
tmp = tmp.reindex(sorted(tmp.columns), axis=1)
tmp = tmp.transpose()
overview_error['CIFAR10'] = tmp[10]

In [49]:
overview_error

,MNIST,FashionMNIST,CIFAR10
FedAvg,0.074547,0.054585,0.043727
FlTrust,0.074287,0.044983,0.050824
Krum,0.092504,0.041928,0.057379
Sentinel,0.082352,0.049930,0.036738
SentinelGlobal,0.077599,0.047320,0.042838
TrimmedMean,0.077734,0.050720,0.035592


In [50]:
pd.set_option('colheader_justify', 'center')
overview_error_style = overview_error.style.set_caption(f'F1 SEM\n')
# overview_error_style.highlight_min(props='font-weight: bold')
#tbl_style = overview_style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
#tbl_style.set_properties(**{'text-align': 'center'}).hide_index()
dfi.export(overview_error_style, "figures/baseline_overview_error.png", dpi=300)